## Polars home page (https://pola.rs/) 
## The User Guide home page (https://docs.pola.rs/)

In [2]:
import polars as pl
import numpy as np

## DataFrame Initialization

Many more details in documentation: [here](https://docs.pola.rs/api/python/stable/reference/dataframe/index.html).

Some simple idioms:

In [13]:
pop_size = 3

# Create a dataframe via a dictionary
population = pl.DataFrame(
                        {
                        "id": range(pop_size),
                        "state": ["Susceptible"] * pop_size,
                        }
                        )
print(population)

# Create the same dataframe via list of pl.Series
population2 = [
                pl.Series("id", range(pop_size), dtype=pl.Int64),
                pl.Series("state",  ["Susceptible"] * pop_size),  
              ]
population2 = pl.DataFrame(population2)
print(population2)

shape: (3, 2)
┌─────┬─────────────┐
│ id  ┆ state       │
│ --- ┆ ---         │
│ i64 ┆ str         │
╞═════╪═════════════╡
│ 0   ┆ Susceptible │
│ 1   ┆ Susceptible │
│ 2   ┆ Susceptible │
└─────┴─────────────┘
shape: (3, 2)
┌─────┬─────────────┐
│ id  ┆ state       │
│ --- ┆ ---         │
│ i64 ┆ str         │
╞═════╪═════════════╡
│ 0   ┆ Susceptible │
│ 1   ┆ Susceptible │
│ 2   ┆ Susceptible │
└─────┴─────────────┘


### with_columns()
Very useful function that adds columns, or updates existing columns (if you give a new column the same name as an existing column).

See [documentation link](https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.with_columns.html).

In [14]:
# add a new column using with_columns
population = population.with_columns(age = pl.lit(30))
print(population)

shape: (3, 3)
┌─────┬─────────────┬─────┐
│ id  ┆ state       ┆ age │
│ --- ┆ ---         ┆ --- │
│ i64 ┆ str         ┆ i32 │
╞═════╪═════════════╪═════╡
│ 0   ┆ Susceptible ┆ 30  │
│ 1   ┆ Susceptible ┆ 30  │
│ 2   ┆ Susceptible ┆ 30  │
└─────┴─────────────┴─────┘


### What if we want to update some rows based on a condition?

pl.when().then().otherwise(), inside a with_columns()

Documentation: [here](https://docs.pola.rs/api/python/stable/reference/expressions/api/polars.when.html).

In [15]:
# change status of individual with id 2 to "Infectious"

population = population.with_columns(
        (pl.when(pl.col("id") == 2)
        .then(pl.lit("Infectious"))
        .otherwise(pl.col("state"))
        .alias("state"))
    )
print(population)

shape: (3, 3)
┌─────┬─────────────┬─────┐
│ id  ┆ state       ┆ age │
│ --- ┆ ---         ┆ --- │
│ i64 ┆ str         ┆ i32 │
╞═════╪═════════════╪═════╡
│ 0   ┆ Susceptible ┆ 30  │
│ 1   ┆ Susceptible ┆ 30  │
│ 2   ┆ Infectious  ┆ 30  │
└─────┴─────────────┴─────┘


## Writing/reading a dataframe to/from a file

Many formats available, including but absolutely not limitted to CSV

See [here](https://docs.pola.rs/api/python/stable/reference/io.html).

In [16]:
# Save a dataframe to a csv file
population.write_csv("population.csv")
# Read a (different!) dataframe from a csv file, for later use
my_population = pl.read_csv("example_population.csv")
print(my_population)

shape: (5, 7)
┌─────┬─────────────┬─────┬───────────┬─────────┬─────────────────┬────────────────┐
│ id  ┆ state       ┆ age ┆ age_group ┆ age_day ┆ stime_infection ┆ stime_recovery │
│ --- ┆ ---         ┆ --- ┆ ---       ┆ ---     ┆ ---             ┆ ---            │
│ i64 ┆ str         ┆ i64 ┆ i64       ┆ i64     ┆ i64             ┆ i64            │
╞═════╪═════════════╪═════╪═══════════╪═════════╪═════════════════╪════════════════╡
│ 0   ┆ Susceptible ┆ 15  ┆ 1         ┆ 364     ┆ -1              ┆ -1             │
│ 1   ┆ Susceptible ┆ 20  ┆ 2         ┆ 21      ┆ -1              ┆ -1             │
│ 2   ┆ Infectious  ┆ 61  ┆ 6         ┆ 211     ┆ 12              ┆ 20             │
│ 3   ┆ Susceptible ┆ 98  ┆ 9         ┆ 301     ┆ -1              ┆ -1             │
│ 4   ┆ Susceptible ┆ 100 ┆ 10        ┆ 2       ┆ -1              ┆ -1             │
└─────┴─────────────┴─────┴───────────┴─────────┴─────────────────┴────────────────┘


## DataFrame Updating

In [17]:
# Update the age structure after a time step of one day

t_step = 1 # day
a_year = 365 # days
tstep_per_year = a_year / t_step
years_age_group = 10 # 10-year age band in each age group

my_population = my_population.with_columns(
                # update (in-place) the age column by adding 0 or 1 based on if it's the birthday of an individual
                pl.col('age').add((pl.col('age_day') + t_step) // a_year).alias('age'), 
                # update (in-place) the fractional ages of everyone
                ((pl.col('age_day') + t_step) % a_year).alias('age_day'),
                # update the age groups of people who reach the next 10-year age-group
                ((pl.col('age') + (pl.col('age_day') + t_step) // a_year) // years_age_group).alias('age_group'))

print(my_population)

shape: (5, 7)
┌─────┬─────────────┬─────┬───────────┬─────────┬─────────────────┬────────────────┐
│ id  ┆ state       ┆ age ┆ age_group ┆ age_day ┆ stime_infection ┆ stime_recovery │
│ --- ┆ ---         ┆ --- ┆ ---       ┆ ---     ┆ ---             ┆ ---            │
│ i64 ┆ str         ┆ i64 ┆ i64       ┆ i64     ┆ i64             ┆ i64            │
╞═════╪═════════════╪═════╪═══════════╪═════════╪═════════════════╪════════════════╡
│ 0   ┆ Susceptible ┆ 16  ┆ 1         ┆ 0       ┆ -1              ┆ -1             │
│ 1   ┆ Susceptible ┆ 20  ┆ 2         ┆ 22      ┆ -1              ┆ -1             │
│ 2   ┆ Infectious  ┆ 61  ┆ 6         ┆ 212     ┆ 12              ┆ 20             │
│ 3   ┆ Susceptible ┆ 98  ┆ 9         ┆ 302     ┆ -1              ┆ -1             │
│ 4   ┆ Susceptible ┆ 100 ┆ 10        ┆ 3       ┆ -1              ┆ -1             │
└─────┴─────────────┴─────┴───────────┴─────────┴─────────────────┴────────────────┘


## What if we want to implement deaths?

One way is to just calculate the daily effective death rate by age, and give everyone a chance to die every timestep.

In [18]:
yearly_death_rate = pl.read_csv("death_rates.csv")

# take yearly death rates and calculate adjusted daily death rates
annual_factor = 1.0 / tstep_per_year
adj_death_rate = yearly_death_rate.with_columns(
    (1 - (1 - pl.col("death_rate")) ** (annual_factor)).alias("adj_death_rate")
)
print(adj_death_rate.head())

# let's join this death rate df to the population df (we'll talk more about joins later)
df_with_rates = my_population.join(adj_death_rate, on='age')
print(df_with_rates)


shape: (5, 3)
┌─────┬────────────┬────────────────┐
│ age ┆ death_rate ┆ adj_death_rate │
│ --- ┆ ---        ┆ ---            │
│ i64 ┆ f64        ┆ f64            │
╞═════╪════════════╪════════════════╡
│ 0   ┆ 0.00322    ┆ 0.000009       │
│ 1   ┆ 0.00023    ┆ 6.3021e-7      │
│ 2   ┆ 0.000125   ┆ 3.4249e-7      │
│ 3   ┆ 0.000105   ┆ 2.8769e-7      │
│ 4   ┆ 0.00009    ┆ 2.4659e-7      │
└─────┴────────────┴────────────────┘
shape: (5, 9)
┌─────┬─────────────┬─────┬───────────┬───┬──────────────┬──────────────┬────────────┬─────────────┐
│ id  ┆ state       ┆ age ┆ age_group ┆ … ┆ stime_infect ┆ stime_recove ┆ death_rate ┆ adj_death_r │
│ --- ┆ ---         ┆ --- ┆ ---       ┆   ┆ ion          ┆ ry           ┆ ---        ┆ ate         │
│ i64 ┆ str         ┆ i64 ┆ i64       ┆   ┆ ---          ┆ ---          ┆ f64        ┆ ---         │
│     ┆             ┆     ┆           ┆   ┆ i64          ┆ i64          ┆            ┆ f64         │
╞═════╪═════════════╪═════╪═══════════╪═══╪══════

In [19]:
notrandom_seed = 115
rng = np.random.default_rng(notrandom_seed)

# Let's go forward a day and see who survives

# Two choices: i) drop those who die from the DataFrame, or ii) change the state of those who die:
# i)
df_with_dropped = df_with_rates.filter(pl.col("adj_death_rate") < rng.random(df_with_rates.height))
    
print(df_with_dropped)

# ii)
rng = np.random.default_rng(notrandom_seed)
df_with_state_changed = df_with_rates.with_columns(
    pl.when(pl.col('adj_death_rate') > rng.random(df_with_rates.height))
    .then(pl.lit("Dead"))
    .otherwise(pl.col("state")).alias("state"))
print(df_with_state_changed)

shape: (4, 9)
┌─────┬─────────────┬─────┬───────────┬───┬──────────────┬──────────────┬────────────┬─────────────┐
│ id  ┆ state       ┆ age ┆ age_group ┆ … ┆ stime_infect ┆ stime_recove ┆ death_rate ┆ adj_death_r │
│ --- ┆ ---         ┆ --- ┆ ---       ┆   ┆ ion          ┆ ry           ┆ ---        ┆ ate         │
│ i64 ┆ str         ┆ i64 ┆ i64       ┆   ┆ ---          ┆ ---          ┆ f64        ┆ ---         │
│     ┆             ┆     ┆           ┆   ┆ i64          ┆ i64          ┆            ┆ f64         │
╞═════╪═════════════╪═════╪═══════════╪═══╪══════════════╪══════════════╪════════════╪═════════════╡
│ 0   ┆ Susceptible ┆ 16  ┆ 1         ┆ … ┆ -1           ┆ -1           ┆ 0.00025    ┆ 6.8502e-7   │
│ 1   ┆ Susceptible ┆ 20  ┆ 2         ┆ … ┆ -1           ┆ -1           ┆ 0.00041    ┆ 0.000001    │
│ 2   ┆ Infectious  ┆ 61  ┆ 6         ┆ … ┆ 12           ┆ 20           ┆ 0.00548    ┆ 0.000015    │
│ 3   ┆ Susceptible ┆ 98  ┆ 9         ┆ … ┆ -1           ┆ -1           ┆ 0.2

## Above "works", but is not very efficient, even though it's in polars!

We could instead compute the "age at death" of each agent at the start of the simulation, rather than generating random numbers for each person every day to check.

Much more efficient!

In [20]:
from additional_functions import pick_ages_at_death_given_ages

rng = np.random.default_rng(notrandom_seed)
ages_at_death = pick_ages_at_death_given_ages('death_rates.csv', my_population['age'].to_numpy(), rng)
print(ages_at_death)

[ 91  89  88 100 100]


# Let's implement some disease spread using polars functionality!

In [24]:
#checking new infections

    
print(my_population)

#assume contacts are homogeneous; 
prob_transmission_per_contact = 0.1
avg_contacts_per_day = 2
current_time = 20
avg_inf_duration = 8 # days

#only consider susceptibles
susceptibles = my_population.filter(pl.col("state") == "Susceptible")

notrandom_seed = 11
rng = np.random.default_rng(notrandom_seed)

if susceptibles.height > 0:
    #at least one susceptible ind in the population

    #calculate infected fraction in contacts of a susceptible individual
    frac_infected_in_contacts = my_population.filter(
    pl.col("state") == "Infectious").height / (my_population.height - 1)
    print("frac_infected_in_contacts: %s" %frac_infected_in_contacts)

    #calculate prob_transmission for a susceptible individual
    prob_transmission = prob_transmission_per_contact * avg_contacts_per_day * frac_infected_in_contacts 
    #identify individuals who will be infected
    will_infected = susceptibles.filter(
        rng.random(susceptibles.height) <= prob_transmission
    )

    dur_infections = rng.exponential(avg_inf_duration, will_infected.height)
    new_infected = will_infected.with_columns(
        pl.lit("Infectious").alias("state"),
        pl.lit(current_time).alias("stime_infection"),
        (pl.lit(current_time + dur_infections)).alias("stime_recovery"),
        )
    print("new infected: %s"%new_infected)

    if new_infected.height > 0:
        #only update the rows of the newly infected population 
        df_with_new_inf = my_population.update(new_infected, on="id", how="left")
        
print("updated population %s"%df_with_new_inf)

shape: (5, 7)
┌─────┬─────────────┬─────┬───────────┬─────────┬─────────────────┬────────────────┐
│ id  ┆ state       ┆ age ┆ age_group ┆ age_day ┆ stime_infection ┆ stime_recovery │
│ --- ┆ ---         ┆ --- ┆ ---       ┆ ---     ┆ ---             ┆ ---            │
│ i64 ┆ str         ┆ i64 ┆ i64       ┆ i64     ┆ i64             ┆ i64            │
╞═════╪═════════════╪═════╪═══════════╪═════════╪═════════════════╪════════════════╡
│ 0   ┆ Susceptible ┆ 16  ┆ 1         ┆ 0       ┆ -1              ┆ -1             │
│ 1   ┆ Susceptible ┆ 20  ┆ 2         ┆ 22      ┆ -1              ┆ -1             │
│ 2   ┆ Infectious  ┆ 61  ┆ 6         ┆ 212     ┆ 12              ┆ 20             │
│ 3   ┆ Susceptible ┆ 98  ┆ 9         ┆ 302     ┆ -1              ┆ -1             │
│ 4   ┆ Susceptible ┆ 100 ┆ 10        ┆ 3       ┆ -1              ┆ -1             │
└─────┴─────────────┴─────┴───────────┴─────────┴─────────────────┴────────────────┘
frac_infected_in_contacts: 0.25
new infected: shape

## Your turn!

Let's implement a recovery "update" step, that checks whether anyone moves between Infectious and Susceptible states in a time step

In [25]:
current_time = 20

# Insert some code here!

# Collect summary statistics

In [43]:
#Let's read an example output file
results = pl.read_csv("example_output.csv")
#The file includes number of people (count) in each state at each time step in each simulation run.
print(results)
#OUR AIM: To calculate final size (proportion of the population infected throughout the outbreak)
#It is assumed that population size is constant (closed population) and
# there are no initially recovered individuals

shape: (4_136, 4)
┌─────────────┬───────┬─────┬────────┐
│ state       ┆ count ┆ t   ┆ run_no │
│ ---         ┆ ---   ┆ --- ┆ ---    │
│ str         ┆ i64   ┆ i64 ┆ i64    │
╞═════════════╪═══════╪═════╪════════╡
│ Susceptible ┆ 499   ┆ 0   ┆ 0      │
│ Exposed     ┆ 0     ┆ 0   ┆ 0      │
│ Infectious  ┆ 1     ┆ 0   ┆ 0      │
│ Recovered   ┆ 0     ┆ 0   ┆ 0      │
│ Susceptible ┆ 499   ┆ 1   ┆ 0      │
│ …           ┆ …     ┆ …   ┆ …      │
│ Recovered   ┆ 278   ┆ 86  ┆ 19     │
│ Susceptible ┆ 221   ┆ 87  ┆ 19     │
│ Exposed     ┆ 0     ┆ 87  ┆ 19     │
│ Infectious  ┆ 0     ┆ 87  ┆ 19     │
│ Recovered   ┆ 279   ┆ 87  ┆ 19     │
└─────────────┴───────┴─────┴────────┘


In [44]:
#let's calculate population size first
pop_size = results.filter((pl.col("run_no") == pl.col("run_no").min()) &
                          ((pl.col("t") == pl.col("t").min())))["count"].sum()
print(pop_size)
#let's calculate the proportion of individuals who are recovered at every time step
recovered = results.filter(pl.col("state") == "Recovered"
                          ).with_columns(pl.col("count")/pop_size)
#let's calculate the final size fraction from each run
final_sizes = recovered.group_by("run_no").agg(pl.col("count").last()
                                              ).rename({"count": "fsize_fraction"})
#let's collect summary of final size from all runs
sum_final_size = final_sizes.select(
                        pl.col("fsize_fraction").mean().name.suffix("_mean"),
                        pl.col("fsize_fraction").quantile(0.025).name.suffix("_025"),
                        pl.col("fsize_fraction").quantile(0.975).name.suffix("_975"),
                    )
print(sum_final_size)

500
shape: (1, 3)
┌─────────────────────┬────────────────────┬────────────────────┐
│ fsize_fraction_mean ┆ fsize_fraction_025 ┆ fsize_fraction_975 │
│ ---                 ┆ ---                ┆ ---                │
│ f64                 ┆ f64                ┆ f64                │
╞═════════════════════╪════════════════════╪════════════════════╡
│ 0.305               ┆ 0.002              ┆ 0.692              │
└─────────────────────┴────────────────────┴────────────────────┘


# Let's play aggregating and summarising a slightly larger file

In [42]:
outcome_df = pl.read_csv('outcomes_csv_for_workshop.csv')

print(f'Number of rows: {outcome_df.height}')
print(f'Columns: {outcome_df.columns}')
print(outcome_df.sample(10)) # show 10 random rows
print(f'Different possible outcomes of infection: {outcome_df['outcome'].unique().to_numpy()}')

# let's add a "months alive" column as these seem to be all kids under the age of 2
outcome_df = outcome_df.with_columns((pl.col('age_at_infection') * 12).alias('months_alive').cast(pl.Int32)).drop('age_at_infection')
print(outcome_df)


Number of rows: 425305
Columns: ['simulation', 'age_at_infection', 'day_of_year_infected', 'outcome']
shape: (10, 4)
┌────────────┬──────────────────┬──────────────────────┬─────────┐
│ simulation ┆ age_at_infection ┆ day_of_year_infected ┆ outcome │
│ ---        ┆ ---              ┆ ---                  ┆ ---     │
│ i64        ┆ f64              ┆ i64                  ┆ str     │
╞════════════╪══════════════════╪══════════════════════╪═════════╡
│ 28         ┆ 0.076712         ┆ 304                  ┆ Not SMC │
│ 24         ┆ 0.210959         ┆ 139                  ┆ GP      │
│ 85         ┆ 1.309589         ┆ 207                  ┆ GP      │
│ 51         ┆ 0.120548         ┆ 327                  ┆ Not SMC │
│ 4          ┆ 0.150685         ┆ 260                  ┆ Not SMC │
│ 12         ┆ 0.09863          ┆ 152                  ┆ H       │
│ 52         ┆ 1.342466         ┆ 322                  ┆ Not SMC │
│ 68         ┆ 1.920548         ┆ 55                   ┆ Not SMC │
│ 2         

In [39]:
# How do we count the number of different outcomes by simulation and months_alive?
# group_by and len!
grouped = outcome_df.group_by(
    'simulation',
    'months_alive',
    'outcome',
).len(name='number')
print(grouped)

shape: (5_802, 4)
┌────────────┬──────────────┬─────────┬────────┐
│ simulation ┆ months_alive ┆ outcome ┆ number │
│ ---        ┆ ---          ┆ ---     ┆ ---    │
│ i64        ┆ i32          ┆ str     ┆ u32    │
╞════════════╪══════════════╪═════════╪════════╡
│ 33         ┆ 11           ┆ GP      ┆ 20     │
│ 56         ┆ 17           ┆ Not SMC ┆ 115    │
│ 38         ┆ 11           ┆ H       ┆ 3      │
│ 56         ┆ 9            ┆ H       ┆ 3      │
│ 6          ┆ 12           ┆ H       ┆ 7      │
│ …          ┆ …            ┆ …       ┆ …      │
│ 27         ┆ 22           ┆ GP      ┆ 20     │
│ 80         ┆ 2            ┆ Not SMC ┆ 553    │
│ 32         ┆ 6            ┆ Not SMC ┆ 194    │
│ 25         ┆ 13           ┆ H       ┆ 5      │
│ 35         ┆ 10           ┆ ED      ┆ 3      │
└────────────┴──────────────┴─────────┴────────┘


In [40]:
# Pivot tables are also easy:
pivotted = grouped.pivot(
    'outcome',
    index=('simulation', 'months_alive'),
    values='number',
).sort('simulation', 'months_alive')
print(pivotted)


shape: (1_440, 8)
┌────────────┬──────────────┬─────┬─────────┬──────┬──────┬──────┬──────┐
│ simulation ┆ months_alive ┆ GP  ┆ Not SMC ┆ H    ┆ ICU  ┆ ED   ┆ F    │
│ ---        ┆ ---          ┆ --- ┆ ---     ┆ ---  ┆ ---  ┆ ---  ┆ ---  │
│ i64        ┆ i32          ┆ u32 ┆ u32     ┆ u32  ┆ u32  ┆ u32  ┆ u32  │
╞════════════╪══════════════╪═════╪═════════╪══════╪══════╪══════╪══════╡
│ 1          ┆ 0            ┆ 43  ┆ 141     ┆ 12   ┆ null ┆ 39   ┆ null │
│ 1          ┆ 1            ┆ 124 ┆ 292     ┆ 19   ┆ null ┆ 77   ┆ null │
│ 1          ┆ 2            ┆ 115 ┆ 277     ┆ 19   ┆ 1    ┆ 56   ┆ null │
│ 1          ┆ 3            ┆ 81  ┆ 221     ┆ 21   ┆ null ┆ 26   ┆ null │
│ 1          ┆ 4            ┆ 57  ┆ 136     ┆ 10   ┆ 1    ┆ 18   ┆ null │
│ …          ┆ …            ┆ …   ┆ …       ┆ …    ┆ …    ┆ …    ┆ …    │
│ 100        ┆ 19           ┆ 30  ┆ 107     ┆ null ┆ null ┆ 2    ┆ null │
│ 100        ┆ 20           ┆ 33  ┆ 111     ┆ 1    ┆ null ┆ 2    ┆ null │
│ 100        ┆ 21   

In [41]:
# Oh no, some data is missing from the pivot table
# This is because in that simulation/age there weren't any of those outcomes
# We can fill them in with a fill_nulls

for col in pivotted.columns:
    pivotted = pivotted.with_columns(pl.col(col).fill_null(0))
print(pivotted)

shape: (1_440, 8)
┌────────────┬──────────────┬─────┬─────────┬─────┬─────┬─────┬─────┐
│ simulation ┆ months_alive ┆ GP  ┆ Not SMC ┆ H   ┆ ICU ┆ ED  ┆ F   │
│ ---        ┆ ---          ┆ --- ┆ ---     ┆ --- ┆ --- ┆ --- ┆ --- │
│ i64        ┆ i32          ┆ u32 ┆ u32     ┆ u32 ┆ u32 ┆ u32 ┆ u32 │
╞════════════╪══════════════╪═════╪═════════╪═════╪═════╪═════╪═════╡
│ 1          ┆ 0            ┆ 43  ┆ 141     ┆ 12  ┆ 0   ┆ 39  ┆ 0   │
│ 1          ┆ 1            ┆ 124 ┆ 292     ┆ 19  ┆ 0   ┆ 77  ┆ 0   │
│ 1          ┆ 2            ┆ 115 ┆ 277     ┆ 19  ┆ 1   ┆ 56  ┆ 0   │
│ 1          ┆ 3            ┆ 81  ┆ 221     ┆ 21  ┆ 0   ┆ 26  ┆ 0   │
│ 1          ┆ 4            ┆ 57  ┆ 136     ┆ 10  ┆ 1   ┆ 18  ┆ 0   │
│ …          ┆ …            ┆ …   ┆ …       ┆ …   ┆ …   ┆ …   ┆ …   │
│ 100        ┆ 19           ┆ 30  ┆ 107     ┆ 0   ┆ 0   ┆ 2   ┆ 0   │
│ 100        ┆ 20           ┆ 33  ┆ 111     ┆ 1   ┆ 0   ┆ 2   ┆ 0   │
│ 100        ┆ 21           ┆ 23  ┆ 134     ┆ 0   ┆ 0   ┆ 2   ┆ 0   │
│ 

## Your turn! What aggregations do you want to do? How would you summarise the data?

In [ ]:
# Write some code here.

# Additional examples and functions

In [4]:
#Do you need to add new rows or combine two dataframes?

#let's consider two simple populations (population and  to combine!
pop_size = 2
population_1 = pl.DataFrame(
                        {
                        "id": range(pop_size),
                        "state": ["Susceptible"] * pop_size,
                        }
                        )

population_2 = pl.DataFrame([
    pl.Series("id", [2], dtype=pl.Int64),
    pl.Series("state",  ["Infectious"]),
])
print(population_2)


#There are multiple ways to combine dataframes! 
#You can use concat, vstack, or extend! They all have pros and cons!
#Reference to: https://stackoverflow.com/questions/71654966/how-can-i-append-or-concatenate-two-dataframes-in-python-polars

# concat creates a new memory slab
new_population_1 = pl.concat([population_1, population_2], rechunk=True)

# vstack do not create a memory copy
new_population_2 = population_1.vstack(population_2)

# extend tries to append in place
population_1.extend(population_2)
print(population_1, new_population_1, new_population_2)

shape: (1, 2)
┌─────┬────────────┐
│ id  ┆ state      │
│ --- ┆ ---        │
│ i64 ┆ str        │
╞═════╪════════════╡
│ 2   ┆ Infectious │
└─────┴────────────┘
shape: (3, 2)
┌─────┬─────────────┐
│ id  ┆ state       │
│ --- ┆ ---         │
│ i64 ┆ str         │
╞═════╪═════════════╡
│ 0   ┆ Susceptible │
│ 1   ┆ Susceptible │
│ 2   ┆ Infectious  │
└─────┴─────────────┘ shape: (3, 2)
┌─────┬─────────────┐
│ id  ┆ state       │
│ --- ┆ ---         │
│ i64 ┆ str         │
╞═════╪═════════════╡
│ 0   ┆ Susceptible │
│ 1   ┆ Susceptible │
│ 2   ┆ Infectious  │
└─────┴─────────────┘ shape: (3, 2)
┌─────┬─────────────┐
│ id  ┆ state       │
│ --- ┆ ---         │
│ i64 ┆ str         │
╞═════╪═════════════╡
│ 0   ┆ Susceptible │
│ 1   ┆ Susceptible │
│ 2   ┆ Infectious  │
└─────┴─────────────┘


In [7]:
#Do you need to rename columns? 
#You can either use rename() or with_columns(alias())!

#rename() approach
population_2 = population_1.rename({"id":"unique_id"})
print(population_2)

#alias approach
population_3 = population_1.with_columns(pl.col("id").alias("unique_id")).drop("id")
print(population_3)

#CAREFUL: using alias to rename changes the order of the columns

shape: (3, 2)
┌───────────┬─────────────┐
│ unique_id ┆ state       │
│ ---       ┆ ---         │
│ i64       ┆ str         │
╞═══════════╪═════════════╡
│ 0         ┆ Susceptible │
│ 1         ┆ Susceptible │
│ 2         ┆ Infectious  │
└───────────┴─────────────┘
shape: (3, 2)
┌─────────────┬───────────┐
│ state       ┆ unique_id │
│ ---         ┆ ---       │
│ str         ┆ i64       │
╞═════════════╪═══════════╡
│ Susceptible ┆ 0         │
│ Susceptible ┆ 1         │
│ Infectious  ┆ 2         │
└─────────────┴───────────┘


## need to combine different dataframes on certain axes? Check join()
https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.join.html

## need sorting? Check sort()
https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.sort.html

## need the number of rows of a dataframe? Check height
https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.height.html

## need to check data types? Check schema()
https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.schema.html

## need to introduce columns with list or dict structures? Check struct()
https://docs.pola.rs/api/python/stable/reference/expressions/api/polars.struct.html


# Still not fast enough?

Lazy dataframes ([documentation link](https://docs.pola.rs/api/python/stable/reference/lazyframe/index.html)) help, but add some development overhead (as you can't "look" and debug as easily)